In [9]:
import os
import pandas as pd

FDS_FOLDER = "FDS project"

# 1) Define the CSV files for each entity
#    Key = Entity name, Value = list of CSV filenames
entities_files = {
    "XOM": [
        "XOM_News_2019.csv",
        "XOM_News_2020.csv",
        "XOM_News_2021.csv",
        "XOM_News_2022.csv",
        "XOM_News_2023.csv",
        "XOM_News_2024.csv",
        "XOM_News_2025.csv"
    ],
    "Chevron": [
        "Chevron_News_2019.csv",
        "Chevron_News_2020.csv",
        "Chevron_News_2021.csv",
        "Chevron_News_2022.csv",
        "Chevron_News_2023.csv",
        "Chevron_News_2024.csv",
        "Chevron_News_2025.csv"
    ],
    "Shell": [
        "Shell_News_2019.csv",
        "Shell_News_2020.csv",
        "Shell_News_2021.csv",
        "Shell_News_2022.csv",
        "Shell_News_2023.csv",
        "Shell_News_2024.csv",
        "Shell_News_2025.csv"
    ],
    "BP": [
        "BP_News_2019.csv",
        "BP_News_2020.csv",
        "BP_News_2021.csv",
        "BP_News_2022.csv",
        "BP_News_2023.csv",
        "BP_News_2024.csv",
        "BP_News_2025.csv"
    ],
    "WTI": [
        "WTI_News_2019.csv",
        "WTI_News_2020.csv",
        "WTI_News_2021.csv",
        "WTI_News_2022.csv",
        "WTI_News_2023.csv",
        "WTI_News_2024.csv",
        "WTI_News_2025.csv"
    ],
    "Brent": [
        "Brent_News_2019.csv",
        "Brent_News_2020.csv",
        "Brent_News_2021.csv",
        "Brent_News_2022.csv",
        "Brent_News_2023.csv",
        "Brent_News_2024.csv",
        "Brent_News_2025.csv"
    ],
    "SPY": [
        "SPY_News_2019.csv",
        "SPY_News_2020.csv",
        "SPY_News_2021.csv",
        "SPY_News_2022.csv",
        "SPY_News_2023.csv",
        "SPY_News_2024.csv",
        "SPY_News_2025.csv"
    ]
}

# 2) A helper function to consolidate & clean files for one entity
def consolidate_and_clean(entity, file_list):
    """
    Reads all CSVs in file_list, concatenates them, removes missing & duplicates,
    parses date strings, reformats to DD:MM:YYYY, and saves a cleaned file.
    Returns stats in a dictionary.
    """
    all_dfs = []
    # Read each CSV for this entity
    for filename in file_list:
        csv_path = os.path.join(FDS_FOLDER, filename)
        if os.path.exists(csv_path):
            print(f"[{entity}] Reading file: {csv_path}")
            df_temp = pd.read_csv(csv_path)
            all_dfs.append(df_temp)
        else:
            print(f"[{entity}] File not found: {csv_path} (skipping)")

    if not all_dfs:
        # No files found for this entity
        return {
            "Entity": entity,
            "Initial": 0,
            "AfterMissing": 0,
            "AfterDup": 0
        }

    # Concatenate
    df_combined = pd.concat(all_dfs, ignore_index=True)
    initial_count = len(df_combined)

    # Drop rows missing crucial columns (title, summary, link)
    df_combined.dropna(subset=["title", "summary", "link"], inplace=True)
    after_missing_count = len(df_combined)

    # Remove duplicates by (title, link)
    dup_count = df_combined.duplicated(subset=["title", "link"]).sum()
    if dup_count > 0:
        df_combined.drop_duplicates(subset=["title", "link"], inplace=True)
    after_dup_count = len(df_combined)

    # --- NEW PART: Parse and reformat the date column ---
    # We'll assume the column is named "published_date".
    # 1) Convert to datetime
    df_combined["published_date"] = pd.to_datetime(
        df_combined["published_date"],
        errors="coerce"
    )
    # 2) Reformat to DD:MM:YYYY
    #    (This will produce strings like "20:03:2020")
    df_combined["published_date"] = df_combined["published_date"].dt.strftime("%Y-%m-%d")

    # Save cleaned file
    output_filename = f"{entity}_News_cleaned.csv"
    output_path = os.path.join(FDS_FOLDER, output_filename)
    df_combined.to_csv(output_path, index=False)
    print(f"[{entity}] Cleaned file saved to: {output_path}")
    
    # Return stats
    return {
        "Entity": entity,
        "Initial": initial_count,
        "AfterMissing": after_missing_count,
        "AfterDup": after_dup_count
    }

# 3) Consolidate and clean for each entity, collect stats
stats_list = []
for entity, file_list in entities_files.items():
    stats = consolidate_and_clean(entity, file_list)
    stats_list.append(stats)

# 4) Create a summary table (DataFrame) of the stats
df_stats = pd.DataFrame(stats_list, columns=["Entity", "Initial", "AfterMissing", "AfterDup"])
print("\n=== Summary Table ===")
print(df_stats)

# 5) (Optional) Save the summary table to a CSV
summary_output = os.path.join(FDS_FOLDER, "News_Consolidation_Summary.csv")
df_stats.to_csv(summary_output, index=False)
print(f"\nSummary table saved to: {summary_output}")


[XOM] Reading file: FDS project\XOM_News_2019.csv
[XOM] Reading file: FDS project\XOM_News_2020.csv
[XOM] Reading file: FDS project\XOM_News_2021.csv
[XOM] Reading file: FDS project\XOM_News_2022.csv
[XOM] Reading file: FDS project\XOM_News_2023.csv
[XOM] Reading file: FDS project\XOM_News_2024.csv
[XOM] Reading file: FDS project\XOM_News_2025.csv
[XOM] Cleaned file saved to: FDS project\XOM_News_cleaned.csv
[Chevron] Reading file: FDS project\Chevron_News_2019.csv
[Chevron] Reading file: FDS project\Chevron_News_2020.csv
[Chevron] Reading file: FDS project\Chevron_News_2021.csv
[Chevron] Reading file: FDS project\Chevron_News_2022.csv
[Chevron] Reading file: FDS project\Chevron_News_2023.csv
[Chevron] Reading file: FDS project\Chevron_News_2024.csv
[Chevron] Reading file: FDS project\Chevron_News_2025.csv
[Chevron] Cleaned file saved to: FDS project\Chevron_News_cleaned.csv
[Shell] Reading file: FDS project\Shell_News_2019.csv
[Shell] Reading file: FDS project\Shell_News_2020.csv
[She

In [18]:
import os
import pandas as pd

# Folder with your CSV files
FDS_FOLDER = "FDS project"

# CSV files to process
csv_files = [
    "XOM_data.csv",
    "CVX_data.csv",
    "BP_data.csv",
    "SHEL_data.csv",
    "SPY_data.csv"
]

# Final column names in the exact order we want
final_columns = ["Date", "Close", "High", "Low", "Open", "Volume"]

for filename in csv_files:
    input_path = os.path.join(FDS_FOLDER, filename)
    print(f"Processing {input_path}...")

    # 1) Read CSV, skip first 2 lines (unwanted headers)
    #    so line 3 becomes data
    # 2) Use header=None so we can rename columns ourselves
    df = pd.read_csv(input_path, skiprows=2, header=None)

    # 3) Force the columns to the final names
    #    Make sure the CSV after skipping lines has exactly 6 columns
    df.columns = final_columns

    # 4) Build output filename (e.g. "XOM_data_cleaned.csv")
    output_filename = filename.replace(".csv", "_cleaned.csv")
    output_path = os.path.join(FDS_FOLDER, output_filename)

    # 5) Save the cleaned CSV
    df.to_csv(output_path, index=False)

    print(f"Renamed columns to: {final_columns}")
    print(f"Saved cleaned file to: {output_path}\n")


Processing FDS project\XOM_data.csv...
Renamed columns to: ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
Saved cleaned file to: FDS project\XOM_data_cleaned.csv

Processing FDS project\CVX_data.csv...
Renamed columns to: ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
Saved cleaned file to: FDS project\CVX_data_cleaned.csv

Processing FDS project\BP_data.csv...
Renamed columns to: ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
Saved cleaned file to: FDS project\BP_data_cleaned.csv

Processing FDS project\SHEL_data.csv...
Renamed columns to: ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
Saved cleaned file to: FDS project\SHEL_data_cleaned.csv

Processing FDS project\SPY_data.csv...
Renamed columns to: ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
Saved cleaned file to: FDS project\SPY_data_cleaned.csv

